In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Input
import warnings
warnings.filterwarnings('ignore')

2024-08-01 14:37:22.569919: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 14:37:22.570744: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 14:37:22.574528: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 14:37:22.586930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 14:37:22.611219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [2]:
df = pd.read_csv('bodyfat.csv')

In [3]:
df.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [4]:
df.isnull().sum()

Density    0
BodyFat    0
Age        0
Weight     0
Height     0
Neck       0
Chest      0
Abdomen    0
Hip        0
Thigh      0
Knee       0
Ankle      0
Biceps     0
Forearm    0
Wrist      0
dtype: int64

In [5]:
columns = list(df.columns)
columns.remove('BodyFat')

X = df[columns]
Y = df['BodyFat']

In [6]:
#standardizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [8]:
model = Sequential([
    Input(shape = (14,)),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'linear')
])  

2024-08-01 14:37:24.627197: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-01 14:37:24.628241: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,289 (48.00 KB)

 Trainable params: 12,289 (48.00 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
optimizer = keras.optimizers.Adam(learning_rate = 0.1)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])

In [11]:
epochs = 500
batch_size = 32
h = model.fit(X_train, Y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.1)

Epoch 1/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 957.8459 - mean_squared_error: 957.8459 - val_loss: 192.7431 - val_mean_squared_error: 192.7431
Epoch 2/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 188.6588 - mean_squared_error: 188.6588 - val_loss: 186.6144 - val_mean_squared_error: 186.6144
Epoch 3/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 376.9845 - mean_squared_error: 376.9845 - val_loss: 385.0308 - val_mean_squared_error: 385.0308
Epoch 4/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 280.2759 - mean_squared_error: 280.2759 - val_loss: 81.1021 - val_mean_squared_error: 81.1021
Epoch 5/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 102.3649 - mean_squared_error: 102.3649 - val_loss: 162.0462 - val_mean_squared_error: 162.0462
Epoch 6/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 158.5995 - mean_squared_error: 158.5995 - val_loss: 125.4875 - val_mean_squared_error: 125.4875
Epoch 7/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 88.1862 - mean_square

In [12]:
loss, error = model.evaluate(X_test, Y_test, batch_size=batch_size)
print("Test Loss:", loss)
print("Test Mean Squared Error:", error)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6373 - mean_squared_error: 0.6373 
Test Loss: 0.5783591866493225
Test Mean Squared Error: 0.5783591866493225
